In [15]:
from pyngrok import ngrok
ngrok.set_auth_token("39hlJNDgDaQbCi7nqxFzOs1HqEv_64YxA7yfnuwPRd8svnR4E")



In [16]:
import os
import warnings
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
import uuid, shutil

from app.metadata.schema import MetadataInput
from app.disease.schema import ThyroidInput
from app.metadata.model import predict_metadata
from app.disease.model import predict_thyroid
from app.segmentation.model import segment_image

app = FastAPI(title="Medical AI API")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

UPLOAD_DIR = "uploads"
os.makedirs(UPLOAD_DIR, exist_ok=True)

def safe_return(obj):
    if isinstance(obj, dict):
        return {k: float(v) if hasattr(v, "item") else v for k, v in obj.items()}
    elif hasattr(obj, "tolist"):  
        return obj.tolist()
    elif hasattr(obj, "item"):  
        return obj.item()
    else:
        return str(obj)

# @app.post("/predict/metadata")
# def metadata_predict(data: MetadataInput):
#     result = predict_metadata(data.dict())
#     return safe_return(result)

@app.post("/predict/disease")
def predict_thyroid_disease(data: ThyroidInput):
    result = predict_thyroid(data.dict())
    return safe_return(result)

@app.post("/predict/detected")
async def segmentation_metadata_predict(
    file: UploadFile = File(...),
    age: int = None,
    gender: int = None,
    FNAC: int = None
):
    img_name = f"{uuid.uuid4()}.png"
    img_path = os.path.join(UPLOAD_DIR, img_name)

    with open(img_path, "wb") as f:
        shutil.copyfileobj(file.file, f)

    seg_result = segment_image(img_path)
    seg_result = safe_return(seg_result)

    tirads_value = seg_result.get("tirads")

    if tirads_value is None:
        return {"error": "Failed to calculate TIRADS"}

    metadata_input = {
        "age": age,
        "gender": gender,
        "TIRADS": tirads_value,
        "FNAC": FNAC
    }

    metadata_result = predict_metadata(metadata_input)
    metadata_result = safe_return(metadata_result)

    return {
        "segmentation": seg_result,
        "metadata_prediction": metadata_result
    }

# @app.post("/predict/segmentation-metadata")
# async def segmentation_metadata_predict(file: UploadFile = File(...)):
#     img_name = f"{uuid.uuid4()}.png"
#     img_path = os.path.join(UPLOAD_DIR, img_name)
#     with open(img_path, "wb") as f:
#         shutil.copyfileobj(file.file, f)
#     seg_result = segment_image(img_path)
#     tirads_value = seg_result.get("tirads")
#     if tirads_value is None:
#         return {"error": "Failed to calculate TIRADS"}
#     metadata_result = predict_metadata({"TIRADS": tirads_value})
#     response = {
#         "segmentation": safe_return(seg_result),
#         "metadata_prediction": safe_return(metadata_result)
#     }
#     return response




In [17]:

public_url = ngrok.connect(8005)
print("Public URL:", public_url)


ERROR:pyngrok.process.ngrok:t=2026-02-15T17:02:42+0200 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to send authentication request: failed to fetch CRL. errors encountered: timed out fetching CRL"


Public URL: NgrokTunnel: "https://gertha-unenclosed-nonopinionatively.ngrok-free.dev" -> "http://localhost:8005"


In [18]:
!uvicorn main.ipynb:app --host 0.0.0.0 --port 8500


Traceback (most recent call last):
  File "/home/hunter/py-global/bin/uvicorn", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/hunter/py-global/lib/python3.12/site-packages/click/core.py", line 1485, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hunter/py-global/lib/python3.12/site-packages/click/core.py", line 1406, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/home/hunter/py-global/lib/python3.12/site-packages/click/core.py", line 1269, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hunter/py-global/lib/python3.12/site-packages/click/core.py", line 824, in invoke
    return callback(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hunter/py-global/lib/python3.12/site-packages/uvicorn/main.py", line 424, in main
    run(
  File "/home/hunter/py-global/lib/python3.12/site-packages/uvi